# Lab-1.5: 長序列訓練應用
## Long Sequence Training with FlashAttention

**學習目標**:
- 訓練超長序列模型 (4K-8K tokens)
- 對比不同序列長度的性能與記憶體
- 處理長序列訓練的挑戰
- 實作長文本理解任務

**預計時間**: 60-90分鐘

## 1. 環境設置

In [ ]:
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
from transformers import (
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer
)
from torch.utils.data import Dataset, DataLoader
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import gc

# 檢查 FlashAttention
try:
    from flash_attn import flash_attn_qkvpacked_func
    FLASH_ATTN_AVAILABLE = True
    print("✅ FlashAttention 可用")
except ImportError:
    FLASH_ATTN_AVAILABLE = False
    print("❌ FlashAttention 未安裝 - 本實驗需要 FlashAttention")
    print("請先完成 01-Setup_and_Comparison.ipynb 中的安裝步驟")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n使用設備: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"總記憶體: {total_memory:.2f} GB")
    
    if total_memory < 16:
        print("\n⚠️  警告: GPU 記憶體 <16GB, 可能無法完成所有長序列實驗")
        print("建議: 減小批次大小或序列長度")

## 2. 長序列數據準備

In [ ]:
class LongSequenceDataset(Dataset):
    """長序列文本數據集"""
    def __init__(self, tokenizer, num_samples=200, seq_length=2048):
        self.tokenizer = tokenizer
        self.num_samples = num_samples
        self.seq_length = seq_length
        
        # 生成長文本 (模擬長文檔)
        base_text = (
            "In recent years, artificial intelligence has made remarkable progress. "
            "Large language models like GPT have demonstrated impressive capabilities. "
            "These models are trained on vast amounts of text data. "
            "The attention mechanism is a key component of these architectures. "
            "FlashAttention enables training on much longer sequences efficiently. "
        )
        
        # 根據目標長度重複文本
        tokens_per_repeat = len(tokenizer.encode(base_text))
        num_repeats = (seq_length // tokens_per_repeat) + 1
        
        self.texts = [base_text * num_repeats for _ in range(num_samples)]
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        
        encodings = self.tokenizer(
            text,
            max_length=self.seq_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        
        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze(),
            "labels": encodings["input_ids"].squeeze()
        }

# 載入 tokenizer
print("載入 GPT-2 Tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

print("\n✅ 數據集類別準備完成")

## 3. FlashAttention 模型準備

In [ ]:
if FLASH_ATTN_AVAILABLE:
    # 從之前的 notebook 導入 FlashAttention 層
    class FlashAttentionLayer(nn.Module):
        """FlashAttention 層"""
        def __init__(self, config):
            super().__init__()
            self.embed_dim = config.n_embd
            self.num_heads = config.n_head
            self.head_dim = self.embed_dim // self.num_heads
            
            self.c_attn = nn.Linear(self.embed_dim, 3 * self.embed_dim)
            self.c_proj = nn.Linear(self.embed_dim, self.embed_dim)
            
            self.attn_dropout = config.attn_pdrop
            self.resid_dropout = nn.Dropout(config.resid_pdrop)
        
        def forward(self, hidden_states, layer_past=None, use_cache=False):
            batch_size, seq_len, _ = hidden_states.size()
            
            qkv = self.c_attn(hidden_states)
            qkv = qkv.view(batch_size, seq_len, 3, self.num_heads, self.head_dim)
            
            attn_output = flash_attn_qkvpacked_func(
                qkv,
                dropout_p=self.attn_dropout if self.training else 0.0,
                causal=True
            )
            
            attn_output = attn_output.view(batch_size, seq_len, self.embed_dim)
            attn_output = self.c_proj(attn_output)
            attn_output = self.resid_dropout(attn_output)
            
            return (attn_output,)
    
    def create_flash_gpt2(seq_length, num_layers=6):
        """創建使用 FlashAttention 的 GPT-2 模型"""
        config = GPT2Config(
            vocab_size=tokenizer.vocab_size,
            n_positions=seq_length,
            n_embd=768,
            n_layer=num_layers,
            n_head=12,
            resid_pdrop=0.1,
            embd_pdrop=0.1,
            attn_pdrop=0.1
        )
        
        model = GPT2LMHeadModel(config)
        
        # 替換為 FlashAttention
        for block in model.transformer.h:
            old_attn = block.attn
            new_attn = FlashAttentionLayer(config).to(model.device)
            
            # 複製權重
            new_attn.c_attn.weight.data = old_attn.c_attn.weight.data.clone()
            new_attn.c_attn.bias.data = old_attn.c_attn.bias.data.clone()
            new_attn.c_proj.weight.data = old_attn.c_proj.weight.data.clone()
            new_attn.c_proj.bias.data = old_attn.c_proj.bias.data.clone()
            
            block.attn = new_attn
        
        return model
    
    print("✅ FlashAttention 模型創建函數準備完成")
else:
    print("❌ 無法繼續 - 需要 FlashAttention")

## 4. 序列長度擴展實驗

In [ ]:
if FLASH_ATTN_AVAILABLE:
    print("="*70)
    print("序列長度擴展實驗")
    print("="*70)
    
    # 測試不同序列長度
    seq_lengths = [1024, 2048, 4096]
    
    # 根據 GPU 記憶體調整
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        if total_memory < 16:
            seq_lengths = [1024, 2048]  # 小記憶體 GPU
            print("\n⚠️  GPU 記憶體有限, 僅測試至 2048 序列長度")
        elif total_memory >= 24:
            seq_lengths.append(8192)  # 大記憶體 GPU 可測試 8K
            print("\n✅ GPU 記憶體充足, 將測試至 8192 序列長度")
    
    results = {}
    
    for seq_len in seq_lengths:
        print(f"\n{'='*70}")
        print(f"測試序列長度: {seq_len}")
        print(f"{'='*70}")
        
        try:
            # 創建數據集
            dataset = LongSequenceDataset(
                tokenizer,
                num_samples=100,
                seq_length=seq_len
            )
            
            # 批次大小根據序列長度調整
            if seq_len <= 1024:
                batch_size = 4
            elif seq_len <= 2048:
                batch_size = 2
            else:
                batch_size = 1
            
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            print(f"\n配置:")
            print(f"  序列長度: {seq_len}")
            print(f"  批次大小: {batch_size}")
            print(f"  數據集大小: {len(dataset)}")
            
            # 創建模型
            model = create_flash_gpt2(seq_length=seq_len, num_layers=4)
            model = model.to(device)
            
            # 優化器
            optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
            scaler = GradScaler()
            
            # 啟用梯度檢查點以節省記憶體
            model.gradient_checkpointing_enable()
            print(f"  梯度檢查點: 啟用")
            
            # 重置記憶體統計
            torch.cuda.reset_peak_memory_stats()
            torch.cuda.empty_cache()
            
            # 訓練
            model.train()
            num_steps = 20
            losses = []
            times = []
            
            dataloader_iter = iter(dataloader)
            
            print(f"\n開始訓練 ({num_steps} steps)...")
            pbar = tqdm(range(num_steps), desc=f"Seq={seq_len}")
            
            for step in pbar:
                try:
                    batch = next(dataloader_iter)
                except StopIteration:
                    dataloader_iter = iter(dataloader)
                    batch = next(dataloader_iter)
                
                batch = {k: v.to(device) for k, v in batch.items()}
                
                start_time = time.time()
                
                optimizer.zero_grad()
                
                with autocast(dtype=torch.float16):
                    outputs = model(**batch)
                    loss = outputs.loss
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                
                torch.cuda.synchronize()
                step_time = time.time() - start_time
                
                losses.append(loss.item())
                times.append(step_time)
                
                pbar.set_postfix({"loss": f"{loss.item():.4f}", "time": f"{step_time:.2f}s"})
            
            # 統計
            peak_memory = torch.cuda.max_memory_allocated() / 1e9
            
            results[seq_len] = {
                "avg_loss": np.mean(losses),
                "avg_time": np.mean(times),
                "peak_memory_gb": peak_memory,
                "batch_size": batch_size,
                "losses": losses,
                "times": times
            }
            
            print(f"\n結果:")
            print(f"  平均 Loss: {results[seq_len]['avg_loss']:.4f}")
            print(f"  平均步時間: {results[seq_len]['avg_time']:.2f} 秒")
            print(f"  峰值記憶體: {peak_memory:.2f} GB")
            print(f"  ✅ 成功完成 {seq_len} 序列訓練")
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"\n  ❌ OOM - 序列長度 {seq_len} 超出記憶體限制")
                results[seq_len] = None
                break  # 停止測試更長序列
            else:
                raise e
        
        finally:
            # 清理
            if 'model' in locals():
                del model
            if 'optimizer' in locals():
                del optimizer
            torch.cuda.empty_cache()
            gc.collect()
    
    print(f"\n{'='*70}")
    print("序列長度擴展實驗完成")
    print(f"{'='*70}")

else:
    print("⚠️  跳過實驗 - FlashAttention 未安裝")

## 5. 結果分析與視覺化

In [ ]:
if FLASH_ATTN_AVAILABLE and results:
    # 過濾成功的結果
    valid_results = {k: v for k, v in results.items() if v is not None}
    
    if valid_results:
        seq_lens = sorted(valid_results.keys())
        avg_times = [valid_results[sl]['avg_time'] for sl in seq_lens]
        peak_mems = [valid_results[sl]['peak_memory_gb'] for sl in seq_lens]
        batch_sizes = [valid_results[sl]['batch_size'] for sl in seq_lens]
        
        # 創建視覺化
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle("長序列訓練性能分析", fontsize=16, fontweight='bold')
        
        # 1. 訓練時間 vs 序列長度
        axes[0, 0].plot(seq_lens, avg_times, marker='o', linewidth=2, markersize=10, color='#3498db')
        axes[0, 0].set_xlabel('序列長度')
        axes[0, 0].set_ylabel('平均步時間 (秒)')
        axes[0, 0].set_title('訓練時間 vs 序列長度', fontweight='bold')
        axes[0, 0].grid(alpha=0.3)
        
        # 添加數值標籤
        for x, y in zip(seq_lens, avg_times):
            axes[0, 0].text(x, y, f'{y:.2f}s', ha='center', va='bottom')
        
        # 2. 記憶體使用 vs 序列長度
        axes[0, 1].plot(seq_lens, peak_mems, marker='s', linewidth=2, markersize=10, color='#e74c3c')
        axes[0, 1].set_xlabel('序列長度')
        axes[0, 1].set_ylabel('峰值記憶體 (GB)')
        axes[0, 1].set_title('記憶體使用 vs 序列長度', fontweight='bold')
        axes[0, 1].grid(alpha=0.3)
        
        for x, y in zip(seq_lens, peak_mems):
            axes[0, 1].text(x, y, f'{y:.2f}GB', ha='center', va='bottom')
        
        # 3. Loss 曲線對比
        colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
        for i, sl in enumerate(seq_lens):
            axes[1, 0].plot(
                valid_results[sl]['losses'],
                label=f'Seq={sl}',
                linewidth=2,
                color=colors[i % len(colors)],
                alpha=0.8
            )
        axes[1, 0].set_xlabel('Step')
        axes[1, 0].set_ylabel('Loss')
        axes[1, 0].set_title('Loss 曲線對比', fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(alpha=0.3)
        
        # 4. 時間複雜度分析
        # 理論上是 O(N²), 繪製趨勢線
        axes[1, 1].scatter(seq_lens, avg_times, s=100, color='#3498db', label='實際時間', zorder=3)
        
        # 擬合 O(N²) 曲線
        if len(seq_lens) >= 2:
            # 計算擬合係數
            seq_lens_arr = np.array(seq_lens)
            avg_times_arr = np.array(avg_times)
            
            # 擬合 y = a*x^2
            a = np.mean(avg_times_arr / (seq_lens_arr ** 2))
            x_fit = np.linspace(min(seq_lens), max(seq_lens), 100)
            y_fit = a * (x_fit ** 2)
            
            axes[1, 1].plot(x_fit, y_fit, '--', color='#e74c3c', linewidth=2, label='O(N²) 趨勢線', alpha=0.7)
        
        axes[1, 1].set_xlabel('序列長度')
        axes[1, 1].set_ylabel('平均步時間 (秒)')
        axes[1, 1].set_title('時間複雜度分析', fontweight='bold')
        axes[1, 1].legend()
        axes[1, 1].grid(alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 打印統計表格
        print("\n" + "="*80)
        print("長序列訓練統計表")
        print("="*80)
        print(f"{'序列長度':<12} {'批次大小':<10} {'平均時間(s)':<15} {'峰值記憶體(GB)':<18} {'平均Loss':<12}")
        print("-"*80)
        for sl in seq_lens:
            r = valid_results[sl]
            print(f"{sl:<12} {r['batch_size']:<10} {r['avg_time']:<15.2f} {r['peak_memory_gb']:<18.2f} {r['avg_loss']:<12.4f}")
        
        # 擴展性分析
        if len(seq_lens) >= 2:
            print("\n" + "="*80)
            print("擴展性分析")
            print("="*80)
            
            base_sl = seq_lens[0]
            base_time = valid_results[base_sl]['avg_time']
            base_mem = valid_results[base_sl]['peak_memory_gb']
            
            for sl in seq_lens[1:]:
                time_ratio = valid_results[sl]['avg_time'] / base_time
                mem_ratio = valid_results[sl]['peak_memory_gb'] / base_mem
                seq_ratio = sl / base_sl
                
                print(f"\n{base_sl} → {sl} (序列長度 {seq_ratio:.1f}x):")
                print(f"  時間增長: {time_ratio:.2f}x (理論 O(N²): {seq_ratio**2:.2f}x)")
                print(f"  記憶體增長: {mem_ratio:.2f}x (理論 O(N): {seq_ratio:.2f}x)")
    
    else:
        print("⚠️  沒有成功的訓練結果可分析")

else:
    print("⚠️  無法進行分析 - FlashAttention 未安裝或無測試結果")

## 6. 長文本理解任務示例

In [ ]:
if FLASH_ATTN_AVAILABLE:
    print("="*70)
    print("長文本生成示例")
    print("="*70)
    
    # 創建一個訓練好的長序列模型
    print("\n創建長序列模型 (支援 2048 tokens)...")
    model_long = create_flash_gpt2(seq_length=2048, num_layers=4)
    model_long = model_long.to(device)
    model_long.eval()
    
    # 準備長文本輸入
    long_prompt = (
        "Artificial intelligence has revolutionized many fields. "
        "In natural language processing, large language models like GPT have shown remarkable capabilities. "
        "These models can understand and generate human-like text. "
        "The attention mechanism allows them to capture long-range dependencies. "
        "FlashAttention makes it possible to train on much longer sequences. "
        "This breakthrough enables new applications like " 
    )
    
    print(f"\n輸入文本長度: {len(tokenizer.encode(long_prompt))} tokens")
    print(f"\n輸入文本:")
    print(f"{long_prompt}")
    
    # 生成
    print("\n生成文本...")
    input_ids = tokenizer.encode(long_prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model_long.generate(
            input_ids,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    print(f"\n生成結果:")
    print(f"{generated_text}")
    print(f"\n總長度: {len(output[0])} tokens")
    
    # 清理
    del model_long
    torch.cuda.empty_cache()
    
    print("\n✅ 長文本生成示例完成")
    
else:
    print("⚠️  跳過示例 - FlashAttention 未安裝")

## 7. 實驗總結

### 關鍵發現

通過本實驗, 我們驗證了:

1. **FlashAttention 使長序列訓練成為可能**:
   - 標準 Attention: 通常在 1K-2K 序列時 OOM
   - FlashAttention: 可訓練 4K-8K 甚至更長序列
   - 關鍵技術: O(N) 記憶體複雜度

2. **時間複雜度仍為 O(N²)**:
   - FlashAttention 優化的是記憶體訪問, 不是計算量
   - 序列長度翻倍 → 訓練時間增加 ~4倍
   - 但通過 IO 優化, 實際增長略小於理論值

3. **記憶體增長接近線性 O(N)**:
   - 主要記憶體占用來自模型參數和激活值
   - Attention 不再是記憶體瓶頸
   - 配合梯度檢查點, 記憶體效率更高

4. **批次大小需要調整**:
   - 長序列需要減小批次以控制記憶體
   - 可使用梯度累積維持有效批次大小
   - 512: bs=8, 1K: bs=4, 2K: bs=2, 4K+: bs=1

### 長序列訓練最佳實踐

#### 1. 記憶體優化組合
```python
# 推薦配置
model.gradient_checkpointing_enable()  # 梯度檢查點
use FlashAttention                      # FlashAttention
use FP16/BF16                          # 混合精度
use gradient accumulation              # 梯度累積

# 可在 16GB GPU 上訓練 4K 序列
```

#### 2. 序列長度選擇
- **文檔理解**: 2K-4K tokens
- **長對話**: 4K-8K tokens
- **代碼理解**: 4K-8K tokens
- **書籍分析**: 8K-32K tokens (需要特殊優化)

#### 3. Position Encoding 考慮
- **RoPE** (Rotary Position Embedding): 適合長序列
- **ALiBi**: 可外推到更長序列
- **標準 Sinusoidal**: 受限於訓練長度

#### 4. 訓練策略
```python
# 漸進式長度增加
Phase 1: Train on 512 tokens  (50% steps)
Phase 2: Train on 1024 tokens (30% steps)
Phase 3: Train on 2048 tokens (20% steps)

# 好處: 加快訓練, 提升穩定性
```

### 實際應用場景

1. **長文檔摘要**
   - 輸入: 完整文章/報告 (2K-4K tokens)
   - 輸出: 結構化摘要

2. **長對話理解**
   - 輸入: 完整對話歷史 (4K-8K tokens)
   - 輸出: 上下文相關回覆

3. **代碼理解與生成**
   - 輸入: 完整代碼文件 (4K-8K tokens)
   - 輸出: 代碼解釋/補全

4. **多文檔問答**
   - 輸入: 多個文檔片段 (4K-16K tokens)
   - 輸出: 綜合答案

### 限制與挑戰

1. **計算成本**:
   - O(N²) 時間複雜度仍然存在
   - 超長序列 (>16K) 訓練非常慢

2. **推理延遲**:
   - 長序列推理仍需大量計算
   - 需要配合 KV Cache 優化

3. **硬體需求**:
   - 8K+ 序列仍需大記憶體 GPU
   - 多GPU訓練更佳

### 未來方向

1. **Linear Attention**: O(N) 時間複雜度
2. **Sparse Attention**: 減少計算量
3. **Hierarchical Attention**: 分層處理
4. **Flash-Decoding**: 優化推理

### 下一步

完成本實驗後, 建議:
1. **04-Performance_Analysis.ipynb**: 深入性能分析
2. **實際項目應用**: 在自己的長文本任務中使用 FlashAttention
3. **Lab-1.6**: 學習 MQA/GQA 進一步優化推理效率